In [1]:
import numpy as np
from cvxopt import matrix, solvers
import matplotlib.pyplot as plt

In [ ]:
# w - sklon přímky
# b - posun přímky
# ξ⁺ - o kolik je bod nad ε
# ξ⁻ - o kolik je bod pod −ε

In [2]:
def create_model_SVR_primary(X, y, C=10.0, eps=1):
    """generuje matice pro knihovnu CVXOPT pro reseni SVR problemu"""
    m, n = X.shape  # zjisti si rozmery
    # m - pocet bodu
    # n - dimenze
    # generovani matic a vektoru pro ucelovou funkci
    # generovani matice P - pro resic vstupuje jako matice prislusne kvadraticke formy
    P = np.zeros((n + 1 + 2 * m, n + 1 + 2 * m))
    for i in range(0, n):
        P[i, i] = 1  # prepis 1 na diagonalu pro hodnoty w
    # generovani vektoru q - vektor s koeficienty pro linearni cast ucelove funkce
    q = C * np.ones(n + 1 + 2 * m)  # linearni clen v ucelove funkci soucet ksi
    q[:n + 1] = 0  # nastav na nulu pozice w a b ve vektoru q
    # generovani omezujicich podminek
    h = np.concatenate((np.zeros((2 * m, 1)), np.zeros((2 * m, 1))))
    G = np.zeros((4 * m, n + 1 + 2 * m))
    for i, (x, y) in enumerate(zip(X, y)):  # pres vsechny datove body
        # prvni omezeni
        G[i, :n] = x  # sloupce odpovidajici w
        G[i, n] = 1.  # sloupec odpovidajici b
        G[i, n + 1 + i] = -1.  # sloupec ksi+
        h[i] = y + eps
        # druhe omezeni
        G[i + m, :n] = -x  # sloupce odpovidajici w
        G[i + m, n] = -1.  # sloupec odpovidajici b
        G[i + m, n + 1 + i + m] = -1.  # sloupec ksi-
        h[i + m] = -y + eps
        # podminky nezapornosti ksi+
        G[i + 2 * m, n + 1 + i] = -1.
        # podminky nezapornosti ksi-
        G[i + 3 * m, n + 1 + i + m] = -1.

    print(f"P:{P.shape}")
    print(f"q:{q.shape}")
    print(f"G:{G.shape}")
    print(f"h:{h.shape}")
    return matrix(P), matrix(q), matrix(G), matrix(h)